In [ ]:
from datetime import datetime, timedelta

import numpy as np
from skyfield import almanac
from skyfield.api import Angle, N, W, Loader, PlanetaryConstants, utc, wgs84
from skyfield.data import iers
from skyfield.framelib import ecliptic_frame
from skyfield.trigonometry import position_angle_of

import magnitude_lib as ml

In [ ]:
latitude = (35 + 58/60 + 10/3600) * N
longitude = (84 + 19/60) * W
obs_date_time = datetime(2013, 10, 18, 22, 0, 0)
obs_date_time = obs_date_time.replace(tzinfo=utc)
obs_midnight = obs_date_time.date()
obs_midnight_next = obs_midnight + timedelta(days=1)

In [ ]:
obs_date_time

In [ ]:
load = Loader("~/skyfield")
url = load.build_url("finals2000A.all")
with load.open(url) as f:
    finals_data = iers.parse_x_y_dut1_from_finals_all(f)
ts = load.timescale(builtin=False)
iers.install_polar_motion_table(ts, finals_data)

In [ ]:
eph = load("de421.bsp")
moon, sun, earth = eph["Moon"], eph["Sun"], eph["Earth"]

In [ ]:
pc = PlanetaryConstants()
pc.read_text(load('moon_080317.tf'))
pc.read_text(load('pck00008.tpc'))
pc.read_binary(load('moon_pa_de421_1900-2050.bpc'))
moon_frame = pc.build_frame_named('MOON_ME_DE421')

In [ ]:
location = wgs84.latlon(latitude, longitude)
topos = earth + location
obs_date = ts.utc(obs_date_time)

In [ ]:
print(f"Dublin JD: {obs_date.ut1 - 2415020}")

In [ ]:
elong = almanac.moon_phase(eph, obs_date)
fi = almanac.fraction_illuminated(eph, "Moon", obs_date)
moon_phases = almanac.moon_phases(eph)
rise_and_set = almanac.risings_and_settings(eph, moon, location)
transits = almanac.meridian_transits(eph, moon, location)

In [ ]:
print(f'Moon elongation: {elong} degrees')
print(f"Moon fraction illuminated: {fi}")

In [ ]:
obs_date_future = obs_date + 29.75
tf, yf = almanac.find_discrete(obs_date, obs_date_future, moon_phases)
print(tf.utc_iso())
print(yf)
print([almanac.MOON_PHASES[yi] for yi in yf])

In [ ]:
obs_date_past = obs_date - 29.75
tp, yp = almanac.find_discrete(obs_date_past, obs_date, moon_phases)
print(tp.utc_iso())
print(yp)
print([almanac.MOON_PHASES[yi] for yi in yp])

In [ ]:
next_nm = tf[np.where(yf == 0)[0]]
previous_nm = tp[np.where(yp == 0)]
next_fm = tf[np.where(yf == 2)]
print(f"Moon age: {obs_date - previous_nm[0]} days")
print(f"Time to New Moon: {(next_nm.utc_datetime()[0] - obs_date.utc_datetime()).total_seconds() / 3600} hours")
print(f"Time from Previous New Moon: {(obs_date.utc_datetime() - previous_nm.utc_datetime()[0]).total_seconds() / 3600} hours")
print(f"Time to Full Moon: {(next_fm.utc_datetime()[0] - obs_date.utc_datetime()).total_seconds() / (24 * 3600)} days")

In [ ]:
astrometric = topos.at(obs_date).observe(moon)
apparent = astrometric.apparent()
ra, dec, distance = apparent.radec('date')

In [ ]:
print(f"Moon RA: {ra.hours * 15}")
print(f"Moon Dec: {dec.degrees}")
print(f"Moon distance: {distance.km} km")

In [ ]:
alt, az, _ = apparent.altaz()

In [ ]:
print(f"Moon altitude: {alt.degrees}")
print(f"Moon azimuth: {az.degrees}")

In [ ]:
magnitude = ml.ephem_magnitude(elong.radians, distance.au)
mag2 = ml.ephem_magnitude(np.deg2rad(178.56298828125), 386484.25078267464*6.6845871226706E-9)

In [ ]:
print(f"Moon magnitude: {magnitude:.2f}")
print(f"Moon magnitude2: {mag2:.2f}")

In [ ]:
p = moon.at(obs_date).observe(sun).apparent()
lat, lon, distance = p.frame_latlon(moon_frame)
lon_degrees = (lon.degrees + 180.0) % 360.0 - 180.0
print('Sub-solar latitude: {:.5f} degrees'.format(lat.degrees))
print('Sub-solar longitude: {:.5f} degrees'.format(lon_degrees))

In [ ]:
if lon.degrees <= 90.0:
    colong = 90.0 - lon.degrees
else:
    colong = 450.0 - lon.degrees
print(f"Selenographic colongitude: {colong:.3f}")

In [ ]:
p = (earth - moon).at(obs_date)
lat, lon, distance = p.frame_latlon(moon_frame)
lon_degrees = (lon.degrees + 180.0) % 360.0 - 180.0
print('Libration in latitude: {:.3f} degrees'.format(lat.degrees))
print('Libration in longitude: {:.3f} degrees'.format(lon_degrees))
lib_phase = np.arctan2(lon.radians, lat.radians)
lib_phase += 2.0 * np.pi if lib_phase < 0 else 0.0
print(f'Libration phase angle: {np.rad2deg(lib_phase):.5f} degrees')

In [ ]:
obs_m = ts.utc(obs_midnight)
obs_mn = ts.utc(obs_midnight_next)
t, y = almanac.find_discrete(obs_m, obs_mn, rise_and_set)
for ti, yi in zip(t, y):
    print(ti.utc_iso(), 'Rise' if yi else 'Set')

In [ ]:
tt, yt = almanac.find_discrete(obs_m, obs_mn, transits)
transit = tt[np.where(yt == 1)][0]
print(transit.utc_iso(), "Transit")

In [ ]:
moon_radius = pc.assignments["BODY301_RADII"][0]
apparent_diameter = Angle(radians=np.arcsin(moon_radius / distance.km) * 2.0)
print(f'{apparent_diameter.degrees:.6f} degrees')

In [ ]:
apparent_diameter2 = Angle(radians=np.arcsin(1740. / 386484.25078267464) * 2.0)
print(f'{apparent_diameter2.degrees:.6f} degrees')

In [ ]:
print(moon_radius)

In [ ]:
s = moon.at(obs_date).observe(sun)
e = moon.at(obs_date).observe(earth)
pa = e.separation_from(s)
print(f"Phase Angle: {pa}")

In [ ]:
north_pole = moon + pc.build_latlon_degrees(moon_frame, 90.0, 0.0)
np_app = topos.at(obs_date).observe(north_pole).apparent()
a = np_app.altaz()[:-1]
np_pa = position_angle_of((alt, az), a)
#np_pa_degrees = (np_pa.degrees + 180.0) % 360.0 - 180.0
print(f"PA of North Pole = {np_pa.degrees} degrees")